## Cell Fate Decision Generative Model

The form of our data: 

After processing, we obtain number density fractions of difference cell types. These are as follows:

$n_b$=number density of pluripotent cells (both markers) per unit area (# 'both' cells in box)

$n_r$=density of red cells (Oct4) (# 'red' cells in box)

$n_g$=density of green cells (Sox2) (# 'green' cells in box)

Between two time steps, we assert the following: 

$$\Delta n_b= n_{entering, b} - n_{leaving, b} + n_{dividing, b} -n_{b\rightarrow g } - n_{g\rightarrow b}$$

Where $n_{entering, b}$ denotes the number of pluripotent cells entering the ROI, $n_{leaving, b}$ denotes the number exiting the ROI, $n_{dividing,b}$ denotes the number of pluripotent cells in the ROI that divide, $n_{b\rightarrow g } $ the number of cells that differentiate to green, and $n_{g\rightarrow b}$ the number of cells that differentiate to blue. 

$$\Delta n_g= n_{entering, g} - n_{leaving, g} + n_{dividing, g} +n_{b\rightarrow g } $$

Where $n_{entering, g}$ denotes the number of green cells entering the ROI, $n_{leaving, g}$ denotes the number exiting the ROI, and $n_{dividing,g}$ denotes the number of green cells in the ROI that divide.  

$$\Delta n_r= n_{entering, r} - n_{leaving, r} + n_{dividing, r} +n_{b\rightarrow r } $$

With variables defined as above. 

We can then convert these to differential equations (modulo some scaling factor):


$$\frac{\partial n_b(t,x,y)}{\partial t}=k_{move}\nabla^2 n_b(t,x,y)+k_{division}n_b(t,x,y)-(k_{b\rightarrow g}n_b+k_{b\rightarrow r})n_b(t,x,y)$$
$$\frac{\partial n_g(t,x,y)}{\partial t}=k_{move}\nabla^2 n_g(t,x,y)+k_{division}n_g(t,x,y)+k_{b\rightarrow g}n_b(t,x,y)$$
$$\frac{\partial n_r(t,x,y)}{\partial t}=k_{move}\nabla^2 n_r(t,x,y)+k_{division}n_r(t,x,y)+k_{b\rightarrow r}n_b(t,x,y)$$


Once we include noise, numerically solving this set of differential equations should recapitulate our data. Additionally, we have assumed above that $k_{division}$ is the same for all three cell types, as well as rates of motion. This should hold because these cell types are very similar, and are not readily distinguishable without these tags. 

The rates $k$ above are the average rates; in practice, the number densities will be non-deterministic due to various sources of error including: 

- errors due to cell counting algorithm (stochasticity from the observation)
    - To process our data set, we use a black-box cell counting software from MIT's Broad Institute. While this software is very good, it is still not perfect (and is also not as good as the naked eye). It will make errors due to issues such as image quality and resolution (blurring cell outlines), a wide spread of cell sizes (this type of segmentation algorithm usually does better when cells are relatively uniform in size), and possible overlap between cells. Because this is a mixture of many types of uncertainties, each of which has an unknown distribution. We manually counted the cells in a small subset of our images, then compare our counts against the actual count to determine the error of the computer algorithm. Please read the section "Likelihood function" for more details.
    
- stochasticity of cell behavior
    - In this particular case, researchers have observed that the Sox2 reporter is unstable--some cells will silence their fluorescently tagged Sox2 while maintaining their untagged copy.  We expect this $k_{silence}$ to be significantly smaller than the other values, but we should nonetheless include it. However, it is not clear what would be the distribution of uncertainties originated from this Poisson loss process of cell reporters. Including this uncertainties, the model will present statistical noise, and to be mostly conservative, we may assume the uncertainty to be Gaussian with variance $\sigma_{cells}^2$. The estimation of this variance requires further discussion.
    
    $$ \frac{d n_b^*}{dt} = \frac{d n_b}{dt}-k_{silence}  n_b $$ 
    $$ \frac{d n_g^*}{dt} = \frac{d n_g}{dt}-k_{silence} n_g $$
    $$ \frac{d n_r^*}{dt} = \frac{d n_r}{dt}+k_{silence} n_r $$

    The loss of green reporter will be Poisson process with rate $k_{silence}$. Then expected number of loss will be approximated as $n_{silence}=k_{silence} \pm \sqrt{k_{silence}}$. This uncertainty will depend on $k_{silence}$, which leads to time correlated error. In this analysis, we ignore this loss term.

## Parameter estimation using two different models

### Model 1 (without diffusion)

We first simplify our model and set $k_{move}=0$. Then we have a system of ODEs instead of PDEs:

$$ \frac{d n_b (t,x,y)}{dt}  = k_{division} n_b(t,x,y) - (k_{b\rightarrow g } + k_{b \rightarrow r})n_{b}(t,x,y) $$
$$ \frac{d n_g (t,x,y)}{dt} = k_{division}n_g(t,x,y) + k_{b\rightarrow g }n_{g}(t,x,y) $$
$$ \frac{d n_r (t,x,y)}{dt} = k_{division}n_r(t,x,y) + k_{b\rightarrow r }n_{r}(t,x,y) $$

### Model 2 (with diffusion)

We included back the $k_{move}$ term in order for better infered values of k's. So now our model becomes a system of PDEs,

$$\frac{\partial n_b}{\partial t}=k_{move}\nabla^2 n_b+k_{division}n_b-(k_{b\rightarrow g}n_b+k_{b\rightarrow r})n_b$$
$$\frac{\partial n_g}{\partial t}=k_{move}\nabla^2 n_g+k_{division}n_g+k_{b\rightarrow g}n_b$$
$$\frac{\partial n_r}{\partial t}=k_{move}\nabla^2 n_r+k_{division}n_r+k_{b\rightarrow r}n_b$$

where the $k_{move}$ term is modelled by diffusion, namely the divigence of flow of the cell ($\vec{J}$), i.e. $\nabla\cdot\vec{J}\propto\nabla\cdot\nabla n=\nabla^2 n$.<br/>


## Likelihood function
We noticed that the error term is actually proportional to the cell density such that
$$y_b=m_b(1+\Delta n)=m_b+m_b\Delta n$$
where $y_b$ is the cell density of certain cell type in bin $b$ measured by the cell counting software (and Python), $m_b$ is the true value and $\Delta n$ is the percentage difference between them. The error term $m_b\Delta n$ is changing for different cell density $m_b$. To make our likelihood easier to work with, it is better to have an error term independent of the cell density. To do this, we can take the logarithm of both sides,
$$\ln y_b=\ln m_b+\ln(1+\Delta n)$$
in which the error term is $\ln(1+\Delta n)$ that is not dependent on $m_b$.<br/>

To find the mean and variance of $\ln (1+\Delta n)$, we counted cell number in 20 bins of the cell image at time 0 and compared it with the values measured by the Python. We found that the distribution of $\ln (1+\Delta n)$ is roughly Gaussian with mean -0.15 and standard derivation $\approx$ 0.104
<img src="cellfate/result/Vaiance.png" width="400" height="400">

So now the probability to get $\ln y_b$ in the measurement is
$$p(\ln y_b|M,\vec{k},I)=\frac{1}{\sqrt{2\pi\sigma^2}}\exp(-\frac{(\ln y_b-\ln m_b-\mu)^2}{2\sigma^2})$$
where $\mu=-0.15$ and $\sigma=0.104$.<br/>

However, the likelihood of model should be in terms of $p(y)$ instead of $p(\ln y)$. We could do the conversion by considering $$p(y_b)dy_b=p(\ln y_b)d(\ln y_b)=\frac{1}{\sqrt{2\pi\sigma^2}}\exp(-\frac{(\ln y_b-\ln m_b-\mu)^2}{2\sigma^2})\frac{dy_b}{y_b}$$
which implies
$$p(y_b)=\frac{1}{y_b\sqrt{2\pi\sigma^2}}\exp(-\frac{(\ln y_b-\ln m_b-\mu)^2}{2\sigma^2})$$

In this way, the likelihood of our model would be the product of these $p(y_b)$ over all bins $b$,
$$\mathcal{L}=\prod_{b=1}^N p(y_b)$$
where N is the total bins number. Hence the log likelihood would be
$$\ln\mathcal{L}=\ln(\prod_{b=1}^N p(y_b))=\sum_{b=1}^N\ln p(y_b)$$
$$\implies \ln\mathcal{L}=\sum_{b=1}^N \left( -\frac{1}{2}\ln(2\pi\sigma^2)-\ln y_b-\frac{(\ln y_b-\ln m_b-\mu)^2}{2\sigma^2} \right)$$

For the prior, previous experiments indicated that the hourly change of the cell number is much less than 1 per cell per unit area. Also, the values of $k$'s should be non-negative in our setting, meaning that 0 is the lower bound in the prior. Hence we used a uniform prior form 0 to 1. (in contrast to Jeffreys Prior which cannot include 0)

Given above discussion, we are going to estimate parameters using the models with and without the diffusion terms.